In [1]:
import pandas as pd 
import numpy as np 
import mlflow
import ssl
import os

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("GenAI-Course-Simple-RNN-Classifier")
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import tensorflow as tf 
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
max_features = 500
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(25000,) (25000,)
(25000,) (25000,)


In [4]:

max_length = 50
x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)


In [5]:
model = Sequential(
    [
        Embedding(input_dim=max_features, output_dim=128, input_length=max_length),
        SimpleRNN(128, activation="relu"),
        Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 128)           64000     
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 97025 (379.00 KB)
Trainable params: 97025 (379.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2025-04-04 12:02:06.501449: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-04 12:02:06.501470: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-04 12:02:06.501485: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-04 12:02:06.501520: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-04 12:02:06.501536: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss', patience=5,
                         restore_best_weights=True)

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)
mlflow.autolog()
with mlflow.start_run(run_name='movie-review-sentiment-analysis'):
    history = model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=10,
        validation_data=(x_test, y_test),
        callbacks=[callback]
    )

2025/04/04 12:02:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/04/04 12:02:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


Epoch 1/10


2025-04-04 12:02:13.426498: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 79/782 [==>...........................] - ETA: 10:22 - loss: 0.6888 - accuracy: 0.5431

In [ ]:
model.save('movie_review_model.h5')